In [1]:
import os
import dataflows as df
from pprint import pprint
from collections import defaultdict

if not os.environ.get('IMAP_MAIL_USER') or not os.environ.get('IMAP_MAIL_PASSWORD'):
    print('missing IMAP_MAIL_USER or IMAP_MAIL_PASSWORD env vars')

In [2]:
!anyway-etl cbs import-emails

Initializing imap session...
Search imap messages...
Processing 85 messages..
{'bytes': 6655,
 'count_of_rows': 93,
 'dataset_name': None,
 'hash': 'd6a1142459271c5e26c32792495c14c0'}
{'invalid_content_type': 416,
 'invalid_filename': 92,
 'msgIds': 85,
 'valid_files': 93}


In [140]:
stats = defaultdict(int)

def update_stats(row):
    stats['year_{}'.format(row['year'])] += 1

emails = df.Flow(
    df.load('../.data/cbs/emails/datapackage.json'), 
    df.add_computed_field(target='year', operation=lambda row: row['mtime'].year),
    update_stats,
    df.printer(num_rows=1)
).process()
    
pprint(dict(stats))

res_1:
#    msgId    mtime                filename                    filesize     year
     (any)    (datetime)           (string)                   (integer)    (any)
---  -------  -------------------  -----------------------  -----------  -------
1    b'1'     2019-02-07 06:39:43  2019/02/07_06_39_43.zip      1140230     2019
2    b'2'     2019-02-26 13:53:51  2019/02/26_13_53_51.zip      2905508     2019
...
93   b'85'    2021-12-15 15:48:42  2021/12/15_15_48_42.zip      2238841     2021
{'year_2019': 25, 'year_2020': 34, 'year_2021': 34}


In [7]:
!anyway-etl cbs process-files

res_1:
Saving msgId b'85' mtime 2021-12-15 15:48:42 to /home/ori/anyway-etl/.data/cbs/yearly/accidents_type_3/2021
Saving msgId b'84' mtime 2021-12-05 07:07:16 to /home/ori/anyway-etl/.data/cbs/yearly/accidents_type_1/2021
Saving msgId b'62' mtime 2021-02-09 11:26:23 to /home/ori/anyway-etl/.data/cbs/yearly/accidents_type_3/2020
Saving msgId b'68' mtime 2021-05-02 14:09:55 to /home/ori/anyway-etl/.data/cbs/yearly/accidents_type_1/2020
Saving msgId b'39' mtime 2020-05-03 09:23:31 to /home/ori/anyway-etl/.data/cbs/yearly/accidents_type_3/2019
Saving msgId b'70' mtime 2021-05-18 05:03:00 to /home/ori/anyway-etl/.data/cbs/yearly/accidents_type_1/2019
Saving msgId b'8' mtime 2019-04-07 14:23:10 to /home/ori/anyway-etl/.data/cbs/yearly/accidents_type_3/2018
Saving msgId b'45' mtime 2020-06-15 14:23:31 to /home/ori/anyway-etl/.data/cbs/yearly/accidents_type_1/2018
Saving msgId b'44' mtime 2020-06-15 14:19:48 to /home/ori/anyway-etl/.data/cbs/yearly/accidents_type_1/2015
#    msgId    mtime   

In [14]:
df.Flow(
    df.load('../.data/cbs/processed_files/datapackage.json'),
    df.filter_rows(lambda row: row['num_saved_files'] > 0),
    df.printer(num_rows=9999),
).process()

res_1:
  #  msgId    mtime                filename                    filesize  extracted_path         provider_code         year    num_saved_files
     (any)    (datetime)           (string)                   (integer)  (string)                   (integer)    (integer)          (integer)
---  -------  -------------------  -----------------------  -----------  -------------------  ---------------  -----------  -----------------
  1  b'85'    2021-12-15 15:48:42  2021/12/15_15_48_42.zip      2238841  2021/12/15_15_48_42                3         2021                  8
  2  b'84'    2021-12-05 07:07:16  2021/12/05_07_07_16.zip       907564  2021/12/05_07_07_16                1         2021                  8
  3  b'62'    2021-02-09 11:26:23  2021/02/09_11_26_23.zip      2219095  2021/02/09_11_26_23                3         2020                  8
  4  b'68'    2021-05-02 14:09:55  2021/05/02_14_09_55.zip       999922  2021/05/02_14_09_55                1         2020                  8

(<datapackage.package.Package at 0x7f924b1adca0>, {})

In [30]:
!anyway-etl cbs parse-all --load-start-year 2019

Parsing accidents...
load_start_year=2019 output_path=/home/ori/anyway-etl/.data/cbs/accident_markers
cbs_files_dir=/home/ori/anyway-etl/.data/cbs/yearly/accidents_type_1/2019
cbs_files_dir=/home/ori/anyway-etl/.data/cbs/yearly/accidents_type_1/2020
cbs_files_dir=/home/ori/anyway-etl/.data/cbs/yearly/accidents_type_1/2021
cbs_files_dir=/home/ori/anyway-etl/.data/cbs/yearly/accidents_type_3/2019
cbs_files_dir=/home/ori/anyway-etl/.data/cbs/yearly/accidents_type_3/2020
cbs_files_dir=/home/ori/anyway-etl/.data/cbs/yearly/accidents_type_3/2021
{'bytes': 79633901,
 'count_of_rows': 141629,
 'dataset_name': None,
 'hash': '1715c6ea809e8413c6dada3d7003998f'}
{'accidents': 141629, 'valid_directories': 6}
Parsing vehicles...
load_start_year=2019 output_path=/home/ori/anyway-etl/.data/cbs/vehicles
cbs_files_dir=/home/ori/anyway-etl/.data/cbs/yearly/accidents_type_1/2019
cbs_files_dir=/home/ori/anyway-etl/.data/cbs/yearly/accidents_type_1/2020
cbs_files_dir=/home/ori/anyway-etl/.data/cbs/yearly/a

In [31]:
import pandas as pd
from IPython.display import display

for parse_type in ['accident_markers', 'involved', 'vehicles']:
    print('### {} ###'.format(parse_type))
    num_rows_per_provider_year = defaultdict(int)
    top_rows = []

    def get_rows_per_provider_year(row):
        num_rows_per_provider_year['{}_{}'.format(row['__provider_code'], row['__year'])] += 1
        if num_rows_per_provider_year['{}_{}'.format(row['__provider_code'], row['__year'])] <= 2:
            top_rows.append(row)

    df.Flow(
        df.load('../.data/cbs/{}/datapackage.json'.format(parse_type)),
        get_rows_per_provider_year
    ).process()
    
    print('num_rows_per_provider_year:')
    pprint(dict(num_rows_per_provider_year))
    display(pd.DataFrame(top_rows))

### accident_markers ###
num_rows_per_provider_year:
{'1_2019': 12670,
 '1_2020': 10836,
 '1_2021': 9541,
 '3_2019': 44828,
 '3_2020': 31867,
 '3_2021': 31887}


,__provider_code,__year,id,provider_and_id,provider_code,file_type_police,title,description,address,latitude,...,accident_month,accident_day,accident_hour_raw,accident_hour,accident_minute,x,y,vehicle_type_rsa,violation_type_rsa,geom
0,1,2019,2019000014,12019000014,1,None,Accident,"{""REHOV1"": ""\u05d0\u05dc \u05d7\u05e0\u05e1\u0...","אל חנסה, ירושלים",31.814130078339943,...,4,13,50,12,15,221906.0,635701.0,None,None,None
1,1,2019,2019000025,12019000025,1,None,Accident,"{""SHEM_ZOMET"": ""\u05e6\u05d5\u05de\u05ea \u05d...",None,32.271115176105305,...,6,28,57,14,0,200474.0,686393.0,None,None,None
2,1,2020,2019047557,12019047557,1,None,Accident,"{""REHOV1"": ""\u05d3\u05e8\u05da \u05d1\u05e8 \u...","דרך בר אילן, קריית אונו",32.06505086242481,...,1,29,61,15,0,185530.0,663581.0,None,None,None
3,1,2020,2019061987,12019061987,1,None,Accident,"{""REHOV1"": ""\u05d9\u05e4\u05d4 \u05e0\u05d5\u0...","יפה נוף, חדרה",32.431339930929425,...,1,23,30,7,15,194561.0,704174.0,None,None,None
4,1,2021,2020031644,12020031644,1,None,Accident,"{""REHOV1"": ""\u05de\u05e2\u05dc\u05d4 \u05d9\u0...","מעלה יצחק, נצרת עילית",32.70184800537962,...,1,31,73,18,0,230441.0,734148.0,None,None,None
5,1,2021,2020079871,12020079871,1,None,Accident,"{""SUG_DEREH"": 4.0, ""YEHIDA"": 12.0, ""SUG_YOM"": ...",None,33.04566728624023,...,1,14,57,14,0,245802.0,772308.0,None,None,None
6,3,2019,2018024780,32018024780,3,3,Accident,"{""SUG_DEREH"": 4.0, ""YEHIDA"": 38.0, ""SUG_YOM"": ...",None,31.628287309655853,...,1,17,80,19,45,160775.0,615260.0,None,None,None
7,3,2019,2018046239,32018046239,3,3,Accident,"{""SUG_DEREH"": 4.0, ""YEHIDA"": 12.0, ""SUG_YOM"": ...",None,32.96017023157084,...,1,28,45,11,0,209083.0,762796.0,None,None,None
8,3,2020,2019004237,32019004237,3,None,Accident,"{""SUG_DEREH"": 2.0, ""YEHIDA"": 51.0, ""SUG_YOM"": ...",None,32.33332661282791,...,1,4,76,18,45,201495.0,693290.0,None,None,None
9,3,2020,2019005143,32019005143,3,None,Accident,"{""SHEM_ZOMET"": ""\u05e6\u05d5\u05de\u05ea \u05d...",None,31.989513076695104,...,1,5,73,18,0,191317.0,655187.0,None,None,None


### involved ###
num_rows_per_provider_year:
{'1_2019': 33333,
 '1_2020': 26975,
 '1_2021': 24407,
 '3_2019': 101544,
 '3_2020': 71086,
 '3_2021': 71854}


,__provider_code,__year,accident_id,provider_and_id,provider_code,file_type_police,involved_type,license_acquiring_date,age_group,sex,...,home_yishuv_shape,hospital_time,medical_type,release_dest,safety_measures_use,late_deceased,car_id,involve_id,accident_year,accident_month
0,1,2019,2019000014,12019000014,1,None,3,0,16,2,...,12.0,None,None,None,NaN,1.0,0,1,2019,4
1,1,2019,2019000014,12019000014,1,None,1,2016,5,1,...,12.0,None,None,None,NaN,NaN,1,2,2019,4
2,1,2020,2020028982,12020028982,1,None,2,2002,8,2,...,15.0,None,None,None,1.0,NaN,1,2,2020,1
3,1,2020,2020028982,12020028982,1,None,2,1991,11,2,...,31.0,None,None,None,NaN,NaN,2,1,2020,1
4,1,2021,2020031644,12020031644,1,None,3,0,4,2,...,16.0,None,None,None,NaN,NaN,0,2,2021,1
5,1,2021,2020031644,12020031644,1,None,1,2005,8,1,...,25.0,None,None,None,NaN,NaN,1,1,2021,1
6,3,2019,2018024780,32018024780,3,3,2,2012,6,1,...,13.0,None,None,None,NaN,NaN,1,1,2019,1
7,3,2019,2018024780,32018024780,3,3,1,0,99,0,...,NaN,None,None,None,NaN,NaN,2,2,2019,1
8,3,2020,2019004237,32019004237,3,None,1,0,99,0,...,NaN,None,None,None,NaN,NaN,1,2,2020,1
9,3,2020,2019004237,32019004237,3,None,2,2017,6,1,...,26.0,None,None,None,NaN,NaN,2,1,2020,1


### vehicles ###
num_rows_per_provider_year:
{'1_2019': 22773,
 '1_2020': 19498,
 '1_2021': 17413,
 '3_2019': 80747,
 '3_2020': 57481,
 '3_2021': 57933}


,__provider_code,__year,accident_id,provider_and_id,provider_code,file_type_police,engine_volume,manufacturing_year,driving_directions,vehicle_status,vehicle_attribution,vehicle_type,seats,total_weight,car_id,accident_year,accident_month,vehicle_damage
0,1,2019,2019070360,12019070360,1,None,8,2002,1122,0,1,1,1,NaN,1,2019,1,2
1,1,2019,2019074088,12019074088,1,None,1,2018,2211,0,1,8,99,1.0,1,2019,4,2
2,1,2020,2020068206,12020068206,1,None,7,2017,1122,0,1,1,3,2.0,2,2020,6,2
3,1,2020,2020089614,12020089614,1,None,6,2015,4433,0,1,1,3,1.0,1,2020,7,1
4,1,2021,2021005980,12021005980,1,None,4,2020,2211,0,1,1,3,1.0,2,2021,1,1
5,1,2021,2021043950,12021043950,1,None,6,2003,3344,0,1,1,99,1.0,1,2021,2,1
6,3,2019,2019099566,32019099566,3,3,8,2014,0,0,1,1,3,1.0,1,2019,5,4
7,3,2019,2019099566,32019099566,3,3,6,2018,0,0,1,1,3,1.0,2,2019,5,1
8,3,2020,2020056894,32020056894,3,None,5,2015,0,0,1,19,1,NaN,1,2020,1,4
9,3,2020,2020054123,32020054123,3,None,0,0,0,0,1,17,99,0.0,2,2020,2,0
